In [11]:
#stupid jupyter path is in the venv directory, so this is in MTG-INVESTOR-S-GRAIL
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np 
import tensorflow as tf 
from tensorflow.keras.utils import Sequence
import os
import time
import pandas as pd
import Data.sadness as sadness
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
%matplotlib inline
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [12]:
#making file with text from just standard cards
dftemp1 = pd.read_csv('Data/LSTMData.csv', sep=',')
dftemp1 = dftemp1[pd.notnull(dftemp1['oracle_text'])]
with open('Data/LSTMData.txt', 'w', encoding='UTF-8') as fileboi:
    for tupleboi in dftemp1.itertuples():
        if str(tupleboi[2]).count('nan') == 0:
            fileboi.write(str(tupleboi[2])+ '\n')


In [13]:
path2jank = 'Data/LSTMData.txt'
janktext = open(path2jank, 'rb').read().decode(encoding='utf-8')
vocablist = sorted(set(janktext).union(set(sadness.safe_string)))
num_chars = len(vocablist)
char2idx = {u:i for i, u in enumerate(vocablist)}
idx2char = {value:key for key, value in char2idx.items()}

In [14]:
print ('{} unique characters'.format(num_chars))
for char,_ in zip(char2idx, range(num_chars)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

80 unique characters
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '"' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  '+' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '/' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '4' :  16,
  '5' :  17,
  '6' :  18,
  '7' :  19,
  '8' :  20,
  '9' :  21,
  ':' :  22,
  'A' :  23,
  'B' :  24,
  'C' :  25,
  'D' :  26,
  'E' :  27,
  'F' :  28,
  'G' :  29,
  'H' :  30,
  'I' :  31,
  'J' :  32,
  'K' :  33,
  'L' :  34,
  'M' :  35,
  'N' :  36,
  'O' :  37,
  'P' :  38,
  'Q' :  39,
  'R' :  40,
  'S' :  41,
  'T' :  42,
  'U' :  43,
  'V' :  44,
  'W' :  45,
  'X' :  46,
  'Y' :  47,
  'Z' :  48,
  'a' :  49,
  'b' :  50,
  'c' :  51,
  'd' :  52,
  'e' :  53,
  'f' :  54,
  'g' :  55,
  'h' :  56,
  'i' :  57,
  'j' :  58,
  'k' :  59,
  'l' :  60,
  'm' :  61,
  'n' :  62,
  'o' :  63,
  'p' :  64,
  'q' :  65,
  'r' :  66,
  's' :  67,
  't' :  68,
  'u' :  69,
  'v' :  70,
  'w' :  71,
  'x' :  72,
  'y' :  73,
  'z' :  74,
  '{

In [15]:
X_train = []
for tupley in dftemp1.itertuples():
    temparray = np.zeros((len(tupley[2]), num_chars))
    temparray[[range(len(tupley[2]))], [char2idx[c] for c in tupley[2]]] = 1
    X_train.append(temparray)
Y_train = X_train
class MyBatchGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, X, y, batch_size=1, shuffle=True):
        'Initialization'
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.y)/self.batch_size))

    def __getitem__(self, index):
        return self.__data_generation(index)

    def on_epoch_end(self):
        'Shuffles indexes after each epoch'
        self.indexes = np.arange(len(self.y))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, index):
        Xb = np.zeros((self.batch_size, *self.X[index].shape))
        yb = np.zeros((self.batch_size, *self.y[index].shape))
        # naively use the same sample over and over again
        for s in range(0, self.batch_size):
            Xb[s] = self.X[index]
            yb[s] = self.y[index]
        return Xb, yb

In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(num_chars, input_shape=(None, num_chars), return_sequences = True))
model.add(tf.keras.layers.LSTM(50, return_sequences = True))
model.add(tf.keras.layers.LSTM(20, return_sequences = True))
model.add(tf.keras.layers.LSTM(50, return_sequences = True))
model.add(tf.keras.layers.LSTM(num_chars, return_sequences = True))

model.compile(optimizer='adam', loss='categorical_crossentropy')

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, None, 80)          51520     
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 50)          26200     
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 20)          5680      
_________________________________________________________________
lstm_8 (LSTM)                (None, None, 50)          14200     
_________________________________________________________________
lstm_9 (LSTM)                (None, None, 80)          41920     
Total params: 139,520
Trainable params: 139,520
Non-trainable params: 0
_________________________________________________________________


In [18]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [19]:
model.fit_generator(MyBatchGenerator(X_train, Y_train, batch_size=1), epochs=10)

Epoch 1/10
821/821 [==============================] - 676s 824ms/step - loss: 8.3312
Epoch 2/10
566/821 [===================>..........] - ETA: 3:24 - loss: 8.1153